In [4]:
import os

dir = r"C:\Users\maxma\OneDrive\Desktop\Shared Fiber Pho\FiberPhoStuff\data"
bpa = []
for root, dirs, files in os.walk(dir):
        for file in files:
            if file.endswith(".tev"):
                  bpa.append(root)

In [5]:
bpa

['C:\\Users\\maxma\\OneDrive\\Desktop\\Shared Fiber Pho\\FiberPhoStuff\\data\\Vision02-240222-112005',
 'C:\\Users\\maxma\\OneDrive\\Desktop\\Shared Fiber Pho\\FiberPhoStuff\\data\\Vision02-240222-135600']

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from bta import *

ModuleNotFoundError: No module named 'pydantic'

In [ ]:
from os import read
import tdt
data = read_block('C:\\Users\\maxma\\OneDrive\\Desktop\\Shared Fiber Pho\\FiberPhoStuff\\data\\Vision02-240222-112005')
tdt_data = tdt.read_block('C:\\Users\\maxma\\OneDrive\\Desktop\\Shared Fiber Pho\\FiberPhoStuff\\data\\Vision02-240222-112005')
set(tdt_data.epocs["PrtA"].data.tolist())

In [ ]:
prta = data.get_epoc("PrtA")
prtb = data.get_epoc("PrtB")
fi2i = data.get_scalar("Fi2i")
fi2d = data.get_stream("Fi2d")
fi2r = data.get_stream("Fi2r")
gfp = data.get_stream("GFPG")
iso = data.get_stream("ISOG")
prta.onset = tdt_data.epocs.PrtA.onset
prta.offset = tdt_data.epocs.PrtA.offset
prta.data = tdt_data.epocs.PrtA.data
gfp.data = tdt_data.streams.GFPG.data
iso.data = tdt_data.streams.ISOG.data

In [ ]:
from bta._types import TDTEpoc, TDTStream

def get_data(epoc: TDTEpoc, event_code: list[int], gfp: TDTStream, iso: TDTStream, TRANGE: tuple[int, int]):
    trs = get_time_ranges(prtb, values=event_code)
    filtered_csp_trs = modify_time_ranges(epoc, TRANGE, trs)
    filtered_grbda = get_filtered_stream_data(gfp, filtered_csp_trs)
    filtered_iso = get_filtered_stream_data(iso, filtered_csp_trs)
    min1 = np.min([np.size(x) for x in filtered_grbda])
    min2 = np.min([np.size(x) for x in filtered_iso])
    # filtered_grbda = [x[1:min1] for x in filtered_grbda]
    # filtered_iso = [x[1:min2] for x in filtered_iso]

    # Downsample and average 10x via a moving window mean
    N = 10 # Average every 10 samples into 1 value
    F405 = []
    F465 = []
    for lst in filtered_iso:
        small_lst = []
        for i in range(0, min2, N):
            small_lst.append(np.mean(lst[i:i+N-1])) # This is the moving window mean
        F405.append(small_lst)

    for lst in filtered_grbda: 
        small_lst = []
        for i in range(0, min1, N):
            small_lst.append(np.mean(lst[i:i+N-1]))
        F465.append(small_lst)


    #Create a mean signal, standard error of signal, and DC offset
    mean_iso = np.mean(F405, axis=0)
    std_iso = np.std(F405, axis=0)/np.sqrt(len(filtered_iso))
    dc_iso = np.mean(mean_iso)
    mean_gfp = np.mean(F465, axis=0)
    std_gfp = np.std(F465, axis=0)/np.sqrt(len(filtered_grbda))
    dc_gfp = np.mean(mean_gfp)

    # Create the time vector for each stream store
    gfp_vec = TRANGE[0] + np.linspace(1, len(mean_gfp), len(mean_gfp))/gfp.fs*N
    iso_vec = TRANGE[0] + np.linspace(1, len(mean_iso), len(mean_iso))/iso.fs*N

    # Subtract DC offset to get signals on top of one another
    mean_iso = mean_iso - dc_iso
    mean_gfp = mean_gfp - dc_gfp
    
    return mean_iso, mean_gfp, iso_vec, gfp_vec, std_iso, std_gfp



In [ ]:
# # Start making a figure with 4 subplots
# ax0 = fig.add_subplot(5, 1, idx+1)

# # Plotting the traces
# p1, = ax0.plot(ts1, mean_gfp, linewidth=2, color='green', label='GrbDA')
# p2, = ax0.plot(ts2, mean_iso, linewidth=2, color='blueviolet', label='ISO')

# # Plotting standard error bands
# p3 = ax0.fill_between(ts1, mean_gfp+std_gfp, mean_gfp-std_gfp,
#                         facecolor='green', alpha=0.2)
# #p4 = ax0.fill_between(ts2, mean_iso+std_iso, mean_iso-std_iso,
#     #                     facecolor='blueviolet', alpha=0.2)

# # Plotting a line at t = 0 and t = 10
# p5 = ax0.axvline(x=0, linewidth=3, color='slategray', label='Cue Onset')
# p6 = ax0.axvline(x=10, linewidth=3, color='lightblue', label='Reward Delivery')

# # Finish up the plot
# ax0.set_xlabel('Seconds')
# ax0.set_ylabel('Signal (mV)')
# ax0.set_title('GFPG vs ISOS')
# ax0.legend(handles=[p1, p2, p5, p6], loc='upper right')
# # to set range based on minimum and maximum values in sample. created graphs with huge ranges
# # ax0.set_ylim(min(np.min(mean_gfp-std_gfp), np.min(mean_iso-std_iso)),
# #             max(np.max(mean_gfp+std_gfp), np.max(mean_iso+std_iso)))
# ax0.set_ylim(ymin,
#                 ymax)
# ax0.set_xlim(TRANGE[0], TRANGE[1]+TRANGE[0])